In [51]:
import numpy as np

from collections import Counter
from itertools import product

In [52]:
class PauliOperator:
    def __init__(self, matrix, eigenvalues, eigenvectors):
        """
        Bundles the relevant information of a Pauli operator into a single object.

        REMARK: The eigenvalues and eigenvectors are directly passed to avoid numerical inaccuracies.

        :param matrix: The matrix representation of the Pauli operator.
        :param eigenvalues: The eigenvalues of the Pauli operator.
        :param eigenvectors: The eigenvectors of the Pauli operator.
        """
        self.matrix = matrix
        self.eigenvalues = eigenvalues
        self.eigenvectors = eigenvectors


class MultiQubitMeasurementSetting:
    def __init__(self, measurement_settings):
        """
        Represents a multi-qubit measurement setting using Pauli operators.

        :param measurement_settings: A list of Pauli operators representing the measurement setting for each qubit.
        """
        self._measurement_settings = measurement_settings
        self.eigenvectors = []
        self.eigenvalues = []

        self._construct_basis()

    def _construct_basis(self):
        """
        Constructs the multi-qubit basis vectors and their corresponding eigenvalues from the given Pauli operators.
        """

        # in this first loop we iterate over the computational basis states of the n-qubit system
        # constructing them as a list of 0s and 1s allows us to leverage pythons addressing capabilities
        for state_addressing in product(range(2), repeat=len(self._measurement_settings)):
            multi_qubit_eigenvalue = 1.0
            multi_qubit_eigenvector = None

            # we pair up the Pauli operator for each single qubit basis state and retrieve the matching eigenvalue and eigenvector
            for pauli_op, state_index in zip(self._measurement_settings, state_addressing):
                eigenvalue = pauli_op.eigenvalues[state_index]
                eigenvector = pauli_op.eigenvectors[state_index]

                # the eigenvalue of the multi-qubit operator is the product of the single-qubit eigenvalues
                multi_qubit_eigenvalue *= eigenvalue

                # we build up the multi-qubit eigenvector by taking the tensor product of the single-qubit eigenvectors
                if multi_qubit_eigenvector is None:
                    multi_qubit_eigenvector = eigenvector
                else:
                    multi_qubit_eigenvector = np.kron(multi_qubit_eigenvector, eigenvector)

            self.eigenvalues.append(multi_qubit_eigenvalue)
            self.eigenvectors.append(multi_qubit_eigenvector)

In [53]:
sigma_0 = PauliOperator(np.eye(2), [1, 1], [np.array([1, 0]), np.array([0, 1])])
sigma_x = PauliOperator(np.array([[0, 1], [1, 0]]), [1, -1], [np.array([1, 1]) / np.sqrt(2), np.array([1, -1]) / np.sqrt(2)])
sigma_y = PauliOperator(np.array([[0, -1j], [1j, 0]]), [1, -1], [np.array([1, 1j]) / np.sqrt(2), np.array([1, -1j]) / np.sqrt(2)])
sigma_z = PauliOperator(np.array([[1, 0], [0, -1]]), [1, -1], [np.array([1, 0]), np.array([0, 1])])
pauli_operators = { 'I': sigma_0, 'X': sigma_x, 'Y': sigma_y, 'Z': sigma_z }


def sample_state(state, pauli_labels, num_samples=1000):
    measurement_settings = [pauli_operators[pl] for pl in pauli_labels]
    multi_qubit_meas_setting = MultiQubitMeasurementSetting(measurement_settings)

    probs = [np.abs(np.dot(vec, state))**2 for vec in multi_qubit_meas_setting.eigenvectors]

    theo_exp = sum(e * p for e, p in zip(multi_qubit_meas_setting.eigenvalues, probs))

    outcomes = np.random.choice(range(len(state)), size=num_samples, p=probs)
    counts = Counter(outcomes)

    obs_exp = 0
    for state_idx, count_value in counts.items():
        obs_exp += multi_qubit_meas_setting.eigenvalues[state_idx] * (count_value / num_samples)

    return theo_exp, obs_exp

In [54]:
# construct the state

num_qubits = 10

state_dim = 1 << num_qubits                     # bit shifting a ...0001 bitstring is the same as 2**a
psi = np.zeros(state_dim, dtype=complex)      # empty state vector

# since the W state has only non-zero amplitudes for one-hot states, we only need num_qubits random phases
rng = np.random.default_rng(42)
thetas = rng.uniform(0, 2*np.pi, size=num_qubits)

for j in range(num_qubits):
    idx = 1 << (num_qubits - 1 - j)              # find indexing mask via bit shifting

    # apply the phase to the corresponding amplitude coefficient
    psi[idx] = np.exp(1j * thetas[j]) / np.sqrt(num_qubits)

In [55]:
test_measurements = [
    ('Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'Z'),
    ('X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X'),
    ('Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y'),
    ('Z', 'X', 'Y', 'Z', 'X', 'Y', 'Z', 'X', 'Y', 'Z'),
    ('X', 'Y', 'Z', 'X', 'Y', 'Z', 'X', 'Y', 'Z', 'X'),
    ('Z', 'Z', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X'),
    ('Z', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X'),
    ('X', 'X', 'X', 'X', 'X', 'Y', 'X', 'X', 'X', 'X')
]

num_samples = 1000


print("Performing test measurements...")
theo_exp_dict = {}
obs_exp_dict = {}

for pauli_labels in test_measurements:
    theo_exp, obs_exp = sample_state(psi, pauli_labels, num_samples=num_samples)
    theo_exp_dict[pauli_labels] = theo_exp
    obs_exp_dict[pauli_labels] = obs_exp

print("Measurements completed.")

Performing test measurements...
Measurements completed.


In [56]:
print(f"Pauli String | Theoretical Exp | Observed Exp   | Abs. Difference")
print("-" * 70)

differences = []
for pauli_labels_tuple in test_measurements:
    if pauli_labels_tuple in theo_exp_dict and pauli_labels_tuple in obs_exp_dict:
        theo_val = theo_exp_dict[pauli_labels_tuple]
        obs_val = obs_exp_dict[pauli_labels_tuple]
        diff = np.abs(theo_val - obs_val)
        differences.append(diff)

        pauli_str = "".join(pauli_labels_tuple)
        print(f"{pauli_str}   | {theo_val:15.6f} | {obs_val:14.6f} | {diff:15.6f}")


max_diff = np.max(differences)
avg_diff = np.mean(differences)
std_diff = np.std(differences)


print(f"\nNumber of samples per setting: {num_samples}")
print(f"Maximum absolute difference:   {max_diff:.6f}")
print(f"Average absolute difference:   {avg_diff:.6f}")
print(f"Std. Dev. of differences:      {std_diff:.6f}")

Pauli String | Theoretical Exp | Observed Exp   | Abs. Difference
----------------------------------------------------------------------
ZZZZZZZZZZ   |       -1.000000 |      -1.000000 |        0.000000
XXXXXXXXXX   |       -0.000000 |      -0.006000 |        0.006000
YYYYYYYYYY   |       -0.000000 |       0.036000 |        0.036000
ZXYZXYZXYZ   |        0.000000 |       0.086000 |        0.086000
XYZXYZXYZX   |       -0.000000 |      -0.054000 |        0.054000
ZZXXXXXXXX   |       -0.000000 |       0.006000 |        0.006000
ZXXXXXXXXX   |       -0.000000 |      -0.002000 |        0.002000
XXXXXYXXXX   |       -0.000000 |      -0.030000 |        0.030000

Number of samples per setting: 1000
Maximum absolute difference:   0.086000
Average absolute difference:   0.027500
Std. Dev. of differences:      0.028579
